###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Unidad Administrativa Especial de Gestión de Restitución de Tierras Despojadas del Ministerio de Agricultura

In [1]:
import pandas as pd

###Archivo  779c-66kx.csv

Contiene el conteo de las solicitudes de restitución de tierras discriminadas por departamento, municipio y género de la Unidad de Restitución de Tierras

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/779c-66kx.csv?$limit=500000", dtype={'codigodane': str})
df.head()

,departamentodepredio,municipiodelpredio,codigodane,grupoetarioselsolicitante,numerodesolicitantes
0,Amazonas,El Encanto,91263,Hombre,0
1,Amazonas,El Encanto,91263,Mujer,7
2,Amazonas,La Chorrera,91405,Hombre,2
3,Amazonas,La Chorrera,91405,Mujer,2
4,Amazonas,La Pedrera,91407,Hombre,2


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   departamentodepredio       2160 non-null   object
 1   municipiodelpredio         2160 non-null   object
 2   codigodane                 2160 non-null   object
 3   grupoetarioselsolicitante  2160 non-null   object
 4   numerodesolicitantes       2160 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 84.5+ KB


### Eliminación de columnas irrelevantes para el proyecto

In [4]:
relevant_cols = ['codigodane', 'municipiodelpredio', 'departamentodepredio', 'numerodesolicitantes']
land_restitution_claims_minagricultura = df[relevant_cols]
land_restitution_claims_minagricultura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   codigodane            2160 non-null   object
 1   municipiodelpredio    2160 non-null   object
 2   departamentodepredio  2160 non-null   object
 3   numerodesolicitantes  2160 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 67.6+ KB


### Verificar valores nulos

In [5]:
land_restitution_claims_minagricultura.isnull().sum()

,0
codigodane,0
municipiodelpredio,0
departamentodepredio,0
numerodesolicitantes,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [6]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['codigodane', 'municipiodelpredio', 'departamentodepredio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(land_restitution_claims_minagricultura[column].unique())
    print()


Categorías en la columna 'codigodane':
['91263' '91405' '91407' ... '68001' '19075' '95200']

Categorías en la columna 'municipiodelpredio':
['El Encanto' 'La Chorrera' 'La Pedrera' 'La Victoria' 'Leticia'
 'Miriti - Paraná' 'Puerto Arica' 'Puerto Santander' 'Tarapacá'
 'Abejorral' 'Abriaquí' 'Alejandría' 'Amagá' 'Amalfi' 'Andes'
 'Angelópolis' 'Angostura' 'Anza' 'Apartadó' 'Arboletes' 'Argelia'
 'Armenia' 'Barbosa' 'Bello' 'Belmira' 'Betania' 'Betulia' 'Buriticá'
 'Cáceres' 'Caicedo' 'Caldas' 'Campamento' 'Caracolí' 'Caramanta' 'Carepa'
 'Carolina' 'Chigorodó' 'Cisneros' 'Ciudad Bolívar' 'Concepción'
 'Concordia' 'Copacabana' 'Don Matías' 'Ebéjico' 'El Carmen De Viboral'
 'El Santuario' 'Entrerrios' 'Envigado' 'Fredonia' 'Frontino' 'Giraldo'
 'Girardota' 'Gómez Plata' 'Guadalupe' 'Guarne' 'Guatapé' 'Heliconia'
 'Hispania' 'Itaguí' 'Ituango' 'Jardín' 'Jericó' 'La Ceja' 'La Estrella'
 'La Pintada' 'La Unión' 'Liborina' 'Maceo' 'Marinilla' 'MEDELLÍN'
 'Montebello' 'Murindó' 'Mutatá' 'Nar

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [7]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [8]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    land_restitution_claims_minagricultura.loc[:,col] = land_restitution_claims_minagricultura[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

In [9]:
land_restitution_claims_minagricultura.columns

Index(['codigodane', 'municipiodelpredio', 'departamentodepredio',
       'numerodesolicitantes'],
      dtype='object')

### Verificar que los valores en "codigodane" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [10]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigodane" en el df land_restitution_claims_minagricultura

In [11]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
land_restitution_claims_minagricultura.loc[:,'codigodane'] = land_restitution_claims_minagricultura['codigodane'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = land_restitution_claims_minagricultura['codigodane'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [12]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5 4]


In [13]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,codigodane,count,percentage
0,5,1841,0.852315
1,4,319,0.147685


In [14]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = land_restitution_claims_minagricultura[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigodane']])
    print()

Muestra de registros con longitud 5:
  codigodane
0      91263
1      91263
2      91405
3      91405
4      91407

Muestra de registros con longitud 4:
   codigodane
14       5002
15       5002
16       5002
17       5004
18       5021



In [15]:
# Verificar longitudes de 'cod_mupi' igual a 3 y 7 dígitos
cod_muni_3_6_7 = land_restitution_claims_minagricultura[
    (land_restitution_claims_minagricultura['codigodane'].apply(len) == 4)
]
cod_muni_3_6_7['departamentodepredio'].value_counts()

,count
departamentodepredio,
ANTIOQUIA,279
ATLANTICO,40


Nota: Como se aprecia los 'codigodane' con longitudes de 4 dígitos, corresponden a los departamentos de ANTIOQUIA Y ATLANTICO. Estos departamentos tienen codigo que 05 y 08 respectivamente, al parecer en alguna etapa de su generacion el 0 inicial fue suprimido.

 - Adicionar un cero a los codigo_dane de 4 dígitos

In [16]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_4(codigo):
    if len(codigo) == 4:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
land_restitution_claims_minagricultura.loc[:,'codigodane'] = land_restitution_claims_minagricultura['codigodane'].apply(add_zero_if_length_4)

In [17]:
# Verificar cuantos codigo_dane de 4 dígitos quedaron
len(land_restitution_claims_minagricultura[land_restitution_claims_minagricultura['codigodane'].str.len() == 4])

0

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [18]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [19]:
# Comparar listas de códigos
compare_lists(land_restitution_claims_minagricultura['codigodane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en land_restitution_claims_minagricultura", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en land_restitution_claims_minagricultura que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [20]:
land_restitution_claims_minagricultura[land_restitution_claims_minagricultura['codigodane'] == '94663']

,codigodane,municipiodelpredio,departamentodepredio,numerodesolicitantes
791,94663,MAPIRIPANA,GUAINIA,2
2069,94663,MAPIRIPANA,GUAINIA,13


Mapiripana una poblacion del Guania que en la tabla geolocalizada de municipios de la bd figura como corregimiento del municipio BarrancoMinas de Guania. Por lo tanto reemplazamos este codigo por el de Municipio al que pertenece

In [21]:
# Listado de municipios de Guania
dept_mpios_codes[dept_mpios_codes['dept_name'] == 'GUAINIA']

,dept_mpio_code,dept_name,mupio_name
1092,94001,GUAINIA,INIRIDA
1093,94343,GUAINIA,BARRANCOMINAS
1094,94883,GUAINIA,SAN FELIPE
1095,94884,GUAINIA,PUERTO COLOMBIA
1096,94885,GUAINIA,LA GUADALUPE
1097,94886,GUAINIA,CACAHUAL
1098,94887,GUAINIA,PANA PANA
1099,94888,GUAINIA,MORICHAL


In [22]:
land_restitution_claims_minagricultura.loc[:,'codigodane'] = land_restitution_claims_minagricultura['codigodane'].replace( '94663', '94343')

- Verificar nuevamente que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [23]:
# Comparar listas de códigos
compare_lists(land_restitution_claims_minagricultura['codigodane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en land_restitution_claims_minagricultura", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en land_restitution_claims_minagricultura que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [24]:
# Eliminar columnas que ya no se necesitan
columns_to_drop = ['departamentodepredio', 'municipiodelpredio']
land_restitution_claims_minagricultura = land_restitution_claims_minagricultura.drop(columns=columns_to_drop)

In [25]:
# Adicionar columna para trazabilidad de la fuente
land_restitution_claims_minagricultura['source_id'] = 119

In [26]:
land_restitution_claims_minagricultura.columns

Index(['codigodane', 'numerodesolicitantes', 'source_id'], dtype='object')

In [27]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'codigodane': 'dane_code',
    'numerodesolicitantes': 'number_of_applicants',
    'source_id': 'source_id'
}

# Renombrar las columnasland_restitution_claims_minagricultura.rename(columns=translation_map, inplace=True)

In [28]:
#Estructura final del dataset a integrar a la base de datos
land_restitution_claims_minagricultura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   codigodane            2160 non-null   object
 1   numerodesolicitantes  2160 non-null   int64 
 2   source_id             2160 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 50.8+ KB


## Salvar en archivo csv en el drive

In [29]:
# Guardar en archivos CSV en el drive
land_restitution_claims_minagricultura.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/land_restitution_claims_minagricultura.csv', index=False)